# **ACCIDENTS / INCIDENTS IN AVIATION SINCE 1923**

**Bienvenu CHOUPO  
June 18, 2019**

## 1. **INTRODUCTION**

Air transport is in a substancial growth nowadays especially in developping countries (Africa). 
People need to do a lot of things in a small amount of time. The age of speed indeed !  
Plane crash is scarce, but once it happens, it can be very serious. Over the years, there have been some serious cases of aircraft incidents / accidents  that resulted in loss of many lives.
Those accidents happened in various locations around the globe and involved many airlines companies and different types of aircrafts.  
Having a bunch of data today, is it possible for us to look through those data to see for example:
- Which is the company most involved in air accident: this information could help passengers make decision when choosing an airline company.

- Which phases of flight are the most critical? (This information can serve as a training purpose for people involved in air navigation security like me? Yes I'm working in the field of aeronautics and a met instructor.

In this notebook, we will explore the data of accidents from 1923 to 2019, and see which locations were most involved in accidents and cluster their neighborhoods.


## 2. **DATA**

The data have been extracted from the wiki page:
https://en.wikipedia.org/wiki/List_of_aircraft_accidents_and_incidents_resulting_in_at_least_50_fatalities using the BeautifulSoup package. The data cover a period from 1923-12-21 to 2019-03-10.  
**Inclusion criteria**  
Criteria for inclusion require at least 50 fatalities in a single occurrence involving commercial passenger and cargo flights, military passenger and cargo flights, or general aviation flights that have been involved in a ground or mid-air collision with either a commercial or military passenger or cargo flight.  

On the website page, only the names of locations are given. In order to get information on the geographical coordinates of various locations, I will use the geocoder package.  It will not be easy due to the fact that many locations’ names are not written so as to get the geocoder retrieve their coordinates. So I need to refine the names.   
After having all the information, I will the data in a csv file (Air_Accident.csv)

Foursquare location will be used for clustering.


Let's first import all libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# For scrapping tables from the web
from bs4 import BeautifulSoup
import codecs

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Let's ping the website

In [4]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_aircraft_accidents_and_incidents_resulting_in_at_least_50_fatalities').text

Now we Scrape a table from the website using  BeautifulSoup package

In [5]:
soup = BeautifulSoup(website_url,'lxml')
tables = soup.find_all("table", { "class" : "wikitable" })

How many tables are there on the web page?

In [6]:
print('There are ',len(tables),' tables on the website')

There are  6  tables on the website


Table number 3 is the one that interests us. the first two are informaive, they are table keys.

####  2.1. **Extraction of Tables from the website**

On the website we have multiple tables, exactly 6:
We are interested in the first 3:
Table key (deaths, type) which is table[0]
Table key (location, phase of flight)  which is table[1]
List of aircraft accidents and incidents resulting in 50 or more fatalities which is table[2].
table[2] is a little difficult to extract once, so we'll do it in two phases.

In [6]:
#Write a function to parse the rows and the cells and save them in a 2-D array.
def parse(n):
    data = []
    for row in tables[n].find_all("tr"):
        if row and row.find_all('td'):
            data_row = []
            cells = row.find_all('td')
            for cell in cells:
                if cell:
                    data_row.append(cell.get_text().rstrip())            
            data.append(data_row)
    return data

01. Let's extract the table key (Death, type) and name the dataframe Key_death.   
For that, we call the function parse with argument 0

In [7]:
key_death = pd.DataFrame(parse(0),columns = ['Abbreviation', 'Definition','Total']) 
key_death.shape

(13, 3)

In [8]:
key_death.head()

,Abbreviation,Definition,Total
0,Deaths,Tot,Total
1,C,Crew,None
2,P,Passenger,None
3,G,Ground,None
4,N,Notes,None


Let's rearrange row index 7

In [9]:
key_death.iloc[7,:] = [key_death.loc[7,'Definition'], key_death.loc[7,'Total'], 'None']

Let's drop the first row (index=0) and the column 'Total' which is useless here

In [10]:
#drop index 0
key_death = key_death.drop(index=0)
#drop column 'Total'
key_death = key_death.drop(['Total'],axis=1)

In [11]:
key_death

,Abbreviation,Definition
1,C,Crew
2,P,Passenger
3,G,Ground
4,N,Notes
5,†,No survivors
6,1*,Sole survivor
7,COM,Commercial (accident/incident)
8,MIL,Military (accident/incident)
9,INB,Bombing
10,INH,Hijacking


02. Now Let's extract the table key  (location, phase of flight) and name the dataframe Key_location.   
For that, we call the function parse with argument 1

In [12]:
key_location = pd.DataFrame(parse(1),columns = ['Abbreviation', 'Definition','Total']) 
key_location.shape

(14, 3)

In [42]:
key_location

,Abbreviation,Definition,Total
0,Location,(none),< 20 km (12.5 mi)
1,"""off""",< 20 km (12.5 mi) (water impact),None
2,"""near""",20 km (12.5 mi) to 50 km (31 mi),None
3,"""area of""",> 50 km (31 mi),None
4,Phase,STD,Standing
5,TXI,Taxi,None
6,TOF,Take off,None
7,ICL,Initial climb,None
8,ENR,En route,None
9,MNV,Maneuvering,None


Let's rearrange rows with index 0, 4 and 13 and drop column 'total

In [13]:
# Drop indexes
for i in (0, 4, 13):
    key_location.iloc[i,:] = [key_location.loc[i,'Definition'], key_location.loc[i,'Total'], 'None'] 

key_location.head()

,Abbreviation,Definition,Total
0,(none),< 20 km (12.5 mi),None
1,"""off""",< 20 km (12.5 mi) (water impact),None
2,"""near""",20 km (12.5 mi) to 50 km (31 mi),None
3,"""area of""",> 50 km (31 mi),None
4,STD,Standing,None


Column 'Total'is useless. Let's remove it

In [11]:
key_location = key_location.drop(['Total'],axis=1)

Let's now deal with the main table: List of aircraft accidents and incidents resulting in 50 or more fatalities.  
We will make it in two steps.  
First, let's extract the table "df1" of the list of accidents.  
Second, we extract the table "death" of the total number of death on each accident.  
Then we join the two tables to build our main dataframe

In [14]:
#let's extract the table "df1" of the list of accidents. We call the function parse with argument 2 
df1 = pd.DataFrame(parse(2), columns = ['Type','Incident','Aircraft','Location','Phase','Airport','Distance','Date']) 
df1.shape

(548, 8)

In [ ]:
df1.head(20)

In [15]:
# Let's extract the table "death" of the total number of death on each accident.
# We proceed a little bit differently as this table is not shaped the same way as the others.

data = []
for row in tables[2].find_all("tr"): # still on tables[2]
    if row and row.find_all('td'):
        data_row = []
        cells = row.find_all('th') # Here's the difference with the others
        for cell in cells:
            if cell:
                data_row.append(cell.get_text().rstrip())            
        data.append(data_row)
        
death = pd.DataFrame(data, columns = ['Total','Crew','Passenger','Ground','Notes']) 
death.shape

(548, 5)

In [ ]:
death.head()

We can now built our main dataframe "df" by concatenating the two above.

In [17]:
df = pd.concat([df1, death], axis=1)
df.head()

,Type,Incident,Aircraft,Location,Phase,Airport,Distance,Date,Total,Crew,Passenger,Ground,Notes
0,INH,American Airlines Flight 11,Boeing 767-223ER,"usnewyneNew York City, New York, U.S.",ENR[11],,,2001-09-11,"est. 1,700",11,81,"est. 1,600[nb 2]",†
1,INH,United Airlines Flight 175,Boeing 767-222,"usnewyneNew York City, New York, U.S.",ENR[12],,,2001-09-11,"est. 1,000",9,56,est. 900[nb 2],†
2,COM,Pan Am Flight 1736 andKLM Flight 4805,Boeing 747-121 and Boeing 747-206B,"spctTenerife, Spain",TXI/TOF[10][16][17],TFN,,1977-03-27,583,23,560,0,
3,COM,Japan Airlines Flight 123,Boeing 747SR-46,"juUeno, Japan",ENR[18][19],,,1985-08-12,520,15,505,0,
4,COM,Saudi Arabian Flight 763 andKazakhstan Airline...,Boeing 747-168B and Ilyushin Il-76TD,"indicCharkhi Dadri, India",ENR[20][21],,,1996-11-12,349,33,316,0,†


Let's its shape:

In [136]:
df.shape

(548, 15)

Let's find out whether there are duplicate entries or not.

In [18]:
df[df.duplicated()]

,Type,Incident,Aircraft,Location,Phase,Airport,Distance,Date,Total,Crew,Passenger,Ground,Notes


The results show that each row is unique, which is good.

#### 2.2. **Let's search for the location coordinates**

In [19]:
df.Location.head()

0    usnewyneNew York City, New York, U.S.
1    usnewyneNew York City, New York, U.S.
2                      spctTenerife, Spain
3                            juUeno, Japan
4                indicCharkhi Dadri, India
Name: Location, dtype: object

We see that there are some useless letters in lower cases at the beginning of the locations, 
we need to delete them

In [15]:
# We delete the first non capital letters  
v = df.Location
for i in range (len(df)) :
    s = df.Location[i]
    r=len(s)
    j=0
    while s[j].islower():
        v[i] = s[j+1:r]
        j=j+1

# Set the Locations        
df.Location = v

* #### Get the Coordinates

Let's add the columns 'Latitude' and 'Longitude' to the dataframe

In [16]:
# define new empty columns
df['Latitude'] = np.nan
df['Longitude'] = np.nan

* #### We use geopy library to get the latitude and longitude values of Locations.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [17]:
# Create a function getlocation to retrieve the geographic coordinates
def getlocation(my_data): # my_data is a dataframe
    # Initialization
    latitude = my_data.Latitude
    longitude = my_data.Longitude
    # Assign the coordinates
    for i in my_data.index : # formerly for i in range (len(my_data))
        address = my_data.Location[i]
        # Use the Geocode
        try:
            geolocator = Nominatim(user_agent="ny_explorer")
            loc = geolocator.geocode(address)
            latitude[i] = loc.latitude
            longitude[i] = loc.longitude
        except:
            print('Error on Location ',i)
    
    print('Coordinates Found: ',len(my_data[my_data['Latitude'].notnull()].index))
    print('Still missing: ',len(my_data[my_data['Latitude'].isnull()].index))

We call the function getlocation on our dataframe df

In [ ]:
getlocation(df)

We have found 358 locations out of 548. There are 190 missing.

* ### Let's deal with missing values

Let's split our dataframe into two:  
One (df_miss0) with the correct coordinates, the other df_miss1 made of the missing values.
At the end we will just need to merge the dataframes to rebuilt the whole one.

In [19]:
df_miss0 = df[df['Latitude'].notnull()]
df_miss1 = df[df['Latitude'].isnull()]

Let's refine the names of locations in order to get the geocoder have their geographic coordinates

In [20]:
# We remove the first characters until we reach the character 'char' 
# as we will probably repeat this operation several times in order to get the various locations  
# Let's define a function refine()

def refine(database, char):
    for i in database.index :
        try:
            while (database.loc[i,'Location'][0] != char):
                r=len(database.loc[i,'Location']) # La longueur de la chaine de caractères 'Location' ligne i, colonne j
                database.loc[i,'Location'] = database.loc[i,'Location'][1:r]
            if database.loc[i,'Location'][0] == char : # Go to the beginning of the string, no space
                r=len(database.loc[i,'Location'])
                database.loc[i,'Location'] = database.loc[i,'Location'][1:r]
        except:
            print('error on index ',i)
    
    print('Operation complete')

In [ ]:
#Let's call the function on 'df_miss' and 'coma'
refine(df_miss1," ")
# Nota: this operation takes some time to compete

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


We see from the result that there's a problem on indexes 52 and 361. Let's deal with it right away!

In [ ]:
# df.loc[52,'Location'] = "Gulf of Guineaoff Côte d'Ivoire"
# df.loc[361,'Location'] = 'Central America[nb 27]'
# We set the values manually, knowing that the geocoder works well on those values
df_miss1.loc[52,'Location'] = "Côte dIvoire"
df_miss1.loc[361,'Location'] = 'Central America'

Good! We now call the function getlocation on df_miss1 to have the coordinates.

In [ ]:
getlocation(df_miss1)

We have found 95 coordinates, but there are still 96 missing. We split the dataframe again

In [ ]:
df_miss1_0 = df_miss1[df_miss1['Latitude'].notnull()] # with coordinates
df_miss2 = df_miss1[df_miss1['Latitude'].isnull()] # with no coordinates

Let's repeat the previous process again:  

In [ ]:
# Call the function refine
refine(df_miss2," ")

In [ ]:
# call the function getlocation
getlocation(df_miss2)

No new coordinates found. Let's call refine and getlocation again! (Never give up!!!)

In [ ]:
# Call the function refine
refine(df_miss2," ")

In [ ]:
# call the function getlocation
getlocation(df_miss2)

25 Coordinates found and 71 still missing. Let's split the dataframe and repeat the process

In [ ]:
df_miss2_0 = df_miss2[df_miss2['Latitude'].notnull()] # with coordinates
df_miss3 = df_miss2[df_miss2['Latitude'].isnull()] # with no coordinates

In [ ]:
# Refine Location names
refine(df_miss3," ")

In [ ]:
# Get the Coordinates
getlocation(df_miss3)

42 Coordinates found and 29 still missing. Let's split the data one last time

In [ ]:
df_miss3_0 = df_miss3[df_miss3['Latitude'].notnull()] # with coordinates
df_miss4 = df_miss3[df_miss3['Latitude'].isnull()] # with no coordinates

In [ ]:
df_miss4.Location.head()

And we run the process one last time...

In [ ]:
# refine Location names
refine(df_miss4," ")

In [ ]:
# get the coordinates
getlocation(df_miss4)

7 Coordinates found and 22 still missing. Almost all the locations we can not retrieve the coordinates are from Russia (USSR).   
We can deal with those 22 manually.

We split one last time

In [ ]:
df_miss4_0 = df_miss4[df_miss4['Latitude'].notnull()] # with coordinates
df_miss5 = df_miss4[df_miss4['Latitude'].isnull()] # with no coordinates

In [ ]:
df_miss5.Location

In [ ]:
# What are the missing index?
miss_index = df_miss4[df_miss4['Latitude'].isnull()].index

In [ ]:
# Let's recall the original names
for i in miss_index :
    print('Location ', i,' : ', df.loc[i,'Location'])

We will set the names of locations manually, as we have tested what works with the geocoder.

In [ ]:
df_miss5.loc[13,'Location'] = "Moneron Island"
df_miss5.loc[57,'Location'] = "Karimata Strait"
df_miss5.loc[162,'Location'] = "Verkhnyaya Khava"
df_miss5.loc[170,'Location'] = "Adler"
df_miss5.loc[197,'Location'] = "Spassky"
df_miss5.loc[251,'Location'] = "Yelabuga"
df_miss5.loc[260,'Location'] = "Voznesenka"
df_miss5.loc[267,'Location'] = "Parchum"
df_miss5.loc[280,'Location'] = "Kholodnaia Rechka"
df_miss5.loc[290,'Location'] = "Vurnarsky"
df_miss5.loc[340,'Location'] = "Dmitrovsky"
df_miss5.loc[354,'Location'] = "Kurumoch"
df_miss5.loc[370,'Location'] = "Urakhcha"
df_miss5.loc[388,'Location'] = "Anapa"
df_miss5.loc[395,'Location'] = "Maksut"
df_miss5.loc[403,'Location'] = "Kokoshkino"
df_miss5.loc[418,'Location'] = "Port Moresby"
df_miss5.loc[434,'Location'] = "Chebotovka"
df_miss5.loc[455,'Location'] = "Atamanovka"
df_miss5.loc[477,'Location'] = "Kopsa"
df_miss5.loc[517,'Location'] = "Untsukulsky"
df_miss5.loc[520,'Location'] = "Karacharovo"

Now we retrieve the locations using our function getlocation

In [ ]:
# get the coordinates
getlocation(df_miss5)

#### **2.3. Putting it all together**

Great! We now have all our location geographic coordinates.
It's time to merge all our dataframes: df_miss0, df_miss1_0, df_miss2_0, df_miss3_0, df_miss4_0 and df_miss5.  
We will merge one after another.

In [130]:
df1 = df_miss0.merge(df_miss1_0, how = 'outer')

In [135]:
df1.shape

(452, 15)

In [137]:
df2 = df1.merge(df_miss2_0, how = 'outer')
df2.shape

(477, 15)

In [138]:
df3 = df2.merge(df_miss3_0, how = 'outer')
df3.shape

(519, 15)

In [139]:
df4 = df3.merge(df_miss4_0, how = 'outer')
df4.shape

(526, 15)

In [140]:
df5 = df4.merge(df_miss5, how = 'outer')
df5.shape

(548, 15)

df5 is our DATAFRAME with all coordinates!!!  
It's been a long way to this. so let's save this dataframe to a csv file: "Air_Accident.csv".

In [143]:
df5.to_csv('Air_Accident.csv', index = None, header=True)

Great!!!

Let's also save only the coordinates in a csv_file

In [147]:
coord = df5[['Location','Latitude','Longitude']]

In [149]:
coord.to_csv('Air_Accident_coord.csv', index = None, header=True)